In [1]:
!pip install -r requirements.txt -q

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
!apt-get install -y libsndfile1
# librosa dependency

In [2]:
import json
import tensorflow as tf
import importlib
import seaborn as sns
import numpy as np
import librosa
import IPython
print(tf.__version__)
keras = tf.keras

import data_loader
import layers
importlib.reload(data_loader)
importlib.reload(layers)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

2.0.0-beta1


<module 'layers' from '/tf/soundAttGAN/layers/__init__.py'>

In [3]:
from tensorflow.python.client import device_lib

In [4]:
with open('./config/raw_cnn_1.json', mode='r', encoding='utf8') as f:
    config = json.load(f)

print(config)

dl = data_loader.DataLoader(4)
dl.build()
# y_batch, att_batch = next(iter(dl.next_batch()))

{'batch_size': 32, 'y_length': 65536, 'kernel_input': 128, 'filter_input': 4, 'strides_input': 1, 'kernel_output': 128}


In [5]:
audio_iter = dl.next_audio()

In [6]:
enc = layers.Encoder(config)
dec = layers.Decoder(config)

In [7]:
map(lambda x: x[0], audio_iter)

In [8]:
model = keras.models.Sequential(
    [enc, dec]
)

In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1), loss='mae')

In [10]:
d = tf.data.Dataset.from_generator(dl.next_audio, output_types='float32')
dataset = tf.data.Dataset.zip((d, d)).batch(4)

W0814 23:07:19.950817 140632830134080 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [11]:
tape = tf.GradientTape()

In [12]:
tape.gradient()

TypeError: gradient() missing 2 required positional arguments: 'target' and 'sources'

In [13]:
opt = keras.optimizers.Adam(learning_rate=0.01)

In [14]:

for x, _ in dataset:
    
    with tf.GradientTape() as tape:
        x_ = model(x)
        loss = tf.losses.mean_absolute_error(x, x_)
        loss = tf.reduce_sum(loss, axis=1)
        loss = tf.reduce_mean(loss)
#         print(f'loss: {loss.shape}')
        print(f'loss: {loss}')
        
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))
    


loss: 4098.7294921875
loss: 4026.44287109375
loss: 2549.9287109375
loss: 1541.3743896484375
loss: 6854.1123046875
loss: 1543.3831787109375
loss: 2464.07763671875
loss: 1407.251220703125
loss: 1700.8839111328125


KeyboardInterrupt: 

In [21]:
IPython.display.Audio(x[3].numpy().reshape([-1,]), rate=22010)

In [16]:
x_ = model(x)

In [23]:
IPython.display.Audio(x_[2].numpy().reshape([-1,]), rate=22010)